## 할 일
* 훈련 데이터셋, 테스트 데이터셋(밸리데이션 데이터셋) 받기
* 분석, 전처리, 패턴 파악

## 데이터 소개
* Survival  0=사망 1=생존
* pclass    1=어퍼클래스 2=미들클래스 3=로어클래스
* sex       성별
* Age       나이
* sibsp     형제자매 수 + 배우자의 수 (약혼상대 미포함)
* parch     부모 수 + 자녀들 수 (보모 유모 등 미포함)
* ticket    티켓 번호
* fare      요금
* cabin     객실 번호
* embarked  승선한 항구 C = Cherbourg, Q = Queenstown, S = Southampton

In [3]:
# data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# machine learning  #pip install -U scikit-learn
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

ModuleNotFoundError: No module named 'seaborn'

데이터 얻기
* 타이타닉 공식 데이터셋 등 데이터를 폴더에 저장하고 파일입출력한다.

In [ ]:
train_df = pd.read_csv('/home/aimod1/work/pandas/train.csv')
test_df = pd.read_csv('/home/aimod1/work/pandas/test.csv')
combine = [train_df, test_df]

데이터셋에 어떤 Features(특징, 칼럼)가 들어있는지 확인해보기
categorical, nominal, ordinal, ratio, or interval base가 있으며
- categorical 이분법적인
- nominal 이름 데이터
- ordinal 순서를 가지는 데이터
- ratio 비율
- interval 간격

처음에 categorical을 찾아보자

In [ ]:
print(train_df.columns.values)
train_df.head() #첫 줄부터 5개 줄(기본값) 데이터 미리보기 출력

In [ ]:
train_df.tail() #마지막 줄부터 5개 줄(기본값) 데이터 미리보기 출력

* 빈 값, null 값 또는 빈 값을 포함하는 기능은 수정이 필요하다.
* 특히, 객실, 연령 등 데이터에서 많은 null 값이 포함되어 있다.

In [ ]:
train_df.info()
print('_'*40)
test_df.info()

In [ ]:
train_df.describe()
# Review survived rate using `percentiles=[.61, .62]` knowing our problem description mentions 38% survival rate.
# Review Parch distribution using `percentiles=[.75, .8]`
# SibSp distribution `[.68, .69]`
# Age and Fare `[.1, .2, .3, .4, .5, .6, .7, .8, .9, .99]`

In [ ]:
test_df.describe()

In [ ]:
train_df.describe(include=['O']) #categorical한 정보들만 출력시켜본다.

* 상관관계를 알아내는 것이 목표.

In [ ]:
train_df[['Pclass', 'Survived']].groupby(['Pclass'], as_index=False).mean().sort_values(by='Survived', ascending=False) #객실 등급에 따른 생존과의 상관관계

In [ ]:
train_df[["Sex", "Survived"]].groupby(['Sex'], as_index=False).mean().sort_values(by='Survived', ascending=False) #성별에 따른 생존과의 상관관계

In [ ]:
train_df[["SibSp", "Survived"]].groupby(['SibSp'], as_index=False).mean().sort_values(by='SibSp', ascending=True) #자매/배우자에 따른 생존과의 상관관계

In [ ]:
train_df[["Parch", "Survived"]].groupby(['Parch'], as_index=False).mean().sort_values(by='Parch', ascending=True) #부모/자식에 따른 생존과의 상관관계

Observations.관찰.

* Infants (Age <=4) had high survival rate. 4세 이하 영유아들은 생존률이 높았다.
* Oldest passengers (Age = 80) survived. 80세 이상 노인들은 생존률이 높았다.
* Large number of 15-25 year olds did not survive. 15-25세의 대부분들이 사망하였다.
* Most passengers are in 15-35 age range. 승객의 대부분이 15-35세였다.

In [ ]:
g = sns.FacetGrid(train_df, col='Survived', row='Sex', aspect=1.6)
g.map(plt.hist, 'Age', alpha=.5, bins=100)
grid.add_legend();


이번엔 숫자 데이터들끼리 말고 숫자 데이터와 순서가 있는 데이터들끼리 비교해보자.

Observations.

* Pclass=3 had most passengers, however most did not survive. Confirms our classifying assumption #2. 승객의 대다수가 lower클래스였지만 대다수가 사망하였다.
* Infant passengers in Pclass=2 and Pclass=3 mostly survived. Further qualifies our classifying assumption #2. 2,3클래스의 영유아들이 대부분 생존했다.
* Most passengers in Pclass=1 survived. Confirms our classifying assumption #3. 상위클래스의 대부분의 승객이 생존하였다.
* Pclass varies in terms of Age distribution of passengers. 객실 등급은 승객의 연령에 가리지 않고 고르게 분포하였다.

In [ ]:
#grid = sns.FacetGrid(train_df, col='Pclass', hue='Survived')
grid = sns.FacetGrid(train_df, col='Survived', row='Pclass', aspect=1.6)
grid.map(plt.hist, 'Age', alpha=.5, bins=50)
grid.add_legend();

In [ ]:
grid = sns.FacetGrid(train_df, col='Survived', aspect=1.6)
grid.map(plt.hist, 'SibSp', alpha=.5, bins=50)
grid.map(plt.hist, 'Parch', alpha=.5, bins=50)
grid.add_legend();

In [ ]:
# grid = sns.FacetGrid(train_df, col='Embarked')
grid = sns.FacetGrid(train_df, row='Embarked', aspect=1.6)
grid.map(sns.pointplot, 'Pclass', 'Survived', 'Sex', palette='deep')
grid.add_legend()

들어가야 하는 데이터들: []
PassengerId    
* [Survived]    
* [Pclass]    Name    
* [Sex]    
* [Age]    
* [SibSp]    
* [Parch]    Ticket    Fare    Cabin    
* [Embarked]

In [ ]:
"""for dataset in combine:    
    dataset.loc[ dataset['Age'] <= 16, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
    dataset.loc[ dataset['Age'] > 64, 'Age']
train_df.head()"""

In [ ]:
"""성별을 
0=Male과 
1=Female로 나누기
 """

for dataset in combine:
    dataset['Sex'] = dataset['Sex'].map( {'female': 1, 'male': 0} ).astype(int)

train_df.head()

In [ ]:
for dataset in combine:    
    #n = dataset.loc[ dataset['Age'] // 10]
    #dataset.loc[ dataset['Age'], 'Age'] = n
    dataset.loc[ dataset['Age'] < 10, 'Age']
    dataset.loc[(dataset['Age'] >= 10) & (dataset['Age'] < 20), 'Age'] = 1
    dataset.loc[(dataset['Age'] >= 20) & (dataset['Age'] < 30), 'Age'] = 2
    dataset.loc[(dataset['Age'] >= 30) & (dataset['Age'] < 40), 'Age'] = 3
    dataset.loc[(dataset['Age'] >= 40) & (dataset['Age'] < 50), 'Age'] = 4
    dataset.loc[(dataset['Age'] >= 50) & (dataset['Age'] < 60), 'Age'] = 5
    dataset.loc[(dataset['Age'] >= 60) & (dataset['Age'] < 70), 'Age'] = 6
    dataset.loc[(dataset['Age'] >= 70) & (dataset['Age'] < 80), 'Age'] = 7
    dataset.loc[(dataset['Age'] >= 80) & (dataset['Age'] < 90), 'Age'] = 8
train_df.head()